In [172]:
import pandas as pd
import networkx as nx
import ast
import matplotlib.pyplot as plt
import numpy as np

In [225]:
friends_file_path_mask = "../Data/ProcesData/friends/users_from_%s_cluster_friends.csv"
user_info_file_path_mask = "../Data/ProcesData/users_vk_info/users_from_%s_cluster_info.csv"
social_graph_file_path_mask = "../Data/ResultData/graphs/raw_graphs/cluster_%s_social_graph.gexf"
pair_social_graph_file_path_mask = "../Data/ResultData/graphs/pair_graphs/pair_cluster_%s-%s.gexf"

In [276]:
cluster_list = [0,136,45, 53, 137, 123, 105, 129]

In [180]:
chekins = pd.read_csv('../Data/RawData/checkins.csv', encoding='windows-1251')

chekins_group_by_users = pd.DataFrame(chekins.groupby('uid').id.count())
chekins_group_by_users = chekins_group_by_users.rename(index=int, columns={"id": "checkins_count"})
chekins_group_by_users = chekins_group_by_users.reset_index()
checkins_group_by_users['uid'] = checkins_group_by_users.uid.apply(lambda uid: int(float(uid)))

In [181]:
def get_nickname(row):
    if isinstance(row['nickname'], float):
        return row['first_name'] + " " + row['last_name']
    else:
        return row['nickname']

In [219]:
for cluster_num in cluster_list:
    
    user_friends_from_cluster_df = pd.read_csv(friends_file_path_mask % (cluster_num))
    user_friends_from_cluster_df = user_friends_from_cluster_df.drop(['Unnamed: 0'], axis=1)
    
    user_friends_from_cluster_df = user_friends_from_cluster_df.merge(checkins_group_by_users, on='uid', how='left')
    
    users_from_cluster_vk_info_df = pd.read_csv(user_info_file_path_mask % (cluster_num), encoding='windows-1251')
    users_from_cluster_vk_info_df = users_from_cluster_vk_info_df.drop(['Unnamed: 0', 'occupation', 'city', 'country', 'deactivated'], axis=1)
    users_from_cluster_vk_info_df['nickname'] = users_from_cluster_vk_info_df.apply(get_nickname, axis=1)
    
    user_friends_from_cluster_df = user_friends_from_cluster_df.merge(users_from_cluster_vk_info_df, on='uid', how='left')
    
    user_friends_from_cluster_df['friends_list'] =  user_friends_from_cluster_df.friends_list.apply(lambda f_l: ast.literal_eval(f_l))

    user_friends_from_cluster_df = user_friends_from_cluster_df.set_index('uid')
    
    user_friends_from_cluster_df['bdate'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['city_title'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['country_title'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['occupation_type'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['occupation_name'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['domain'].fillna(value='', inplace=True)
    user_friends_from_cluster_df['age'].fillna(value=0, inplace=True)
    user_friends_from_cluster_df['site'].fillna(value='', inplace=True)
    
    nodes = user_friends_from_cluster_df.drop(['friends_list'], axis=1).to_dict('index')
    
    G = nx.Graph()

    for node in nodes:
        G.add_node(node, nodes[node])
    
    for node in G.nodes():
        user_friends_list = user_friends_from_cluster_df.friends_list[node]
        
        for friend_node in user_friends_list:
            if (friend_node in nodes.keys()):
                G.add_edge(node, friend_node)
                
    nx.write_gexf(G, social_graph_file_path_mask % (cluster_num))
    

In [273]:
#get pair graphs

def get_cluster(row):
    if row['in_left_cluster'] and row['in_right_cluster']:
        return 'both'
    elif (row['in_left_cluster']):
        return 'left'
    else:
        return 'right'
    
def get_friends_list(row):
    if row['cluster'] == 'both' or row['cluster'] == 'right':
        return ast.literal_eval(row['friends_list_y'])
    else:
        return ast.literal_eval(row['friends_list_x'])

In [277]:
for left_cluster_index in range(len(cluster_list)):
    for right_cluster_index in range(left_cluster_index, len(cluster_list)):
        if left_cluster_index != right_cluster_index:
            left_cluster_num = cluster_list[left_cluster_index]
            right_cluster_num = cluster_list[right_cluster_index]
            
            left_cluster_users_df = pd.read_csv(friends_file_path_mask % (left_cluster_num))
            left_cluster_users_df['in_left_cluster'] = True
            
            right_cluster_users_df = pd.read_csv(friends_file_path_mask % (right_cluster_num))
            right_cluster_users_df['in_right_cluster'] = True
            
            join_df = pd.merge(left=left_cluster_users_df, right=right_cluster_users_df, on='uid', how='outer')
            
            join_df.in_right_cluster.fillna(value=False, inplace=True)
            join_df.in_left_cluster.fillna(value=False, inplace=True)
            
            join_df['cluster'] = join_df.apply(get_cluster, axis=1)
            
            join_df['friends_list'] = join_df.apply(get_friends_list, axis=1)
            
            join_df = join_df[['uid', 'friends_list', 'cluster']]

            join_df = join_df.set_index('uid')
            
            nodes = join_df.drop(['friends_list'], axis=1).to_dict('index')
            
            G = nx.Graph()

            for node in nodes:
                G.add_node(node, nodes[node])

            for node in G.nodes():
                user_friends_list = join_df.friends_list[node]

                for friend_node in user_friends_list:
                    if (friend_node in nodes.keys()):
                        G.add_edge(node, friend_node)
                        
            
            nx.write_gexf(G, pair_social_graph_file_path_mask % (left_cluster_num, right_cluster_num))

,friends_list,cluster
uid,,
1319766,"[4007, 5923, 9467, 31053, 34836, 40259, 40505,...",left
3409631,"[6596, 76260, 79496, 95000, 96154, 98683, 1008...",left
87902,"[1810, 5822, 8001, 8951, 9172, 18089, 20105, 2...",both
1032245,"[7404, 11635, 11698, 18672, 24329, 49524, 5309...",both
6255541,"[62187, 72821, 79825, 101394, 149182, 166404, ...",left
